In [1]:
import networkx as nx

In [2]:
import torch as th

In [3]:
import networkx
import torch
import numpy as np
import pandas as pd
from sklearn.metrics import *
from torch_geometric.loader import NeighborSampler, NeighborLoader
from torch_geometric.data import Data, DataLoader
from torch_geometric.nn import GATConv, ResGatedGraphConv, GATv2Conv, SAGEConv, GENConv, DeepGCNLayer, PairNorm, GINConv
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
from sklearn.impute import SimpleImputer
import torch.nn.functional as F
from imblearn.under_sampling import RandomUnderSampler
pd.options.mode.use_inf_as_na = True
from collections import Counter
from sklearn.feature_selection import SelectFromModel
import torch.nn as nn
import time
import pickle
from torch.nn import LayerNorm, Linear, ReLU
from torch_scatter import scatter
from tqdm import tqdm
from torch_geometric.loader import RandomNodeSampler
import math
import copy
import dgl
from sklearn.preprocessing import MinMaxScaler, StandardScaler
import random

In [4]:
import dgl
import dgl.nn as dglnn
import torch
import torch.nn as nn
import torch.nn.functional as F
from dgl.data.utils import *

In [5]:
np.random.seed(2022)

In [6]:
import torch_geometric as tg

In [7]:
def get_lg_graph(src_ip, src_port, dst_ip, dst_port, features, label):
    features = features.astype(np.float)
    features = StandardScaler().fit_transform(features)
    features = torch.FloatTensor(features)
    num_nodes = len(label)
    label = label.astype(np.int)
    label = torch.LongTensor(label)
    nodes_to_no_dict = {}
    src_ip = np.array(src_ip)
    dst_ip = np.array(dst_ip)
    src_port = np.array(src_port)
    dst_port = np.array(dst_port)
    src_nodes = list(zip(src_ip, src_port))
    dst_nodes = list(zip(dst_ip, dst_port))
    socket_dict = list(zip(src_nodes, dst_nodes))
    all_nodes = set(src_nodes).union(set(dst_nodes))
    for index,value in enumerate(all_nodes):
        nodes_to_no_dict[value] = index
    src_index = [nodes_to_no_dict[s] for s in src_nodes]
    dst_index = [nodes_to_no_dict[s] for s in dst_nodes]
    index = [src_index, dst_index]
    index = np.array(index)
    g = dgl.graph((src_index, dst_index))
    g.edata['x'] = features
    g.edata['y'] = label
    lg = dgl.line_graph(g, shared=True)
    return lg

In [8]:
def get_data(df):
    ori_data = np.array(df)
    all_index = np.arange(len(df))
    train_mask, test_mask = train_test_split(all_index, stratify=ori_data[:, -1], test_size=0.3, random_state=41)
    x_train, x_test, y_train, y_test = ori_data[train_mask], ori_data[test_mask],ori_data[:, -1][train_mask],ori_data[:, -1][test_mask]
    new_x_test = np.delete(x_test, 6, axis=-1)
    new_x_train = np.delete(x_train, 6, axis=-1)
    test_src_ip = x_test[:, 1]
    test_src_port = x_test[:, 2]
    test_dst_ip = x_test[:, 3]
    test_dst_port = x_test[:, 4]
    test_features = new_x_test[:, 5:-1]
    test_label = x_test[:, -1]
    train_src_ip = x_train[:, 1]
    train_src_port = x_train[:, 2]
    train_dst_ip = x_train[:, 3]
    train_dst_port = x_train[:, 4]
    train_features = new_x_train[:, 5:-1]
    train_label = x_train[:, -1]
    train_lg = get_lg_graph(train_src_ip, train_src_port, train_dst_ip, train_dst_port, train_features, train_label)
    test_lg = get_lg_graph(test_src_ip, test_src_port, test_dst_ip, test_dst_port, test_features, test_label)
    return train_lg, test_lg

In [9]:
df = pd.read_pickle('cic2017_normal_50w.pkl')

In [10]:
train_lg, test_lg = get_data(df)

In [11]:
train_lg, test_lg

(Graph(num_nodes=739589, num_edges=1730061287,
       ndata_schemes={'x': Scheme(shape=(78,), dtype=torch.float32), 'y': Scheme(shape=(), dtype=torch.int64)}
       edata_schemes={}),
 Graph(num_nodes=316967, num_edges=314116663,
       ndata_schemes={'x': Scheme(shape=(78,), dtype=torch.float32), 'y': Scheme(shape=(), dtype=torch.int64)}
       edata_schemes={}))

In [12]:
save_graphs("./dgl_cic2017data50w.bin", [train_lg, test_lg])

In [9]:
data, _ = load_graphs("dgl_cic2017data50w.bin")

In [10]:
data

[Graph(num_nodes=139444, num_edges=1492813740,
       ndata_schemes={'label': Scheme(shape=(), dtype=torch.int64), 'h': Scheme(shape=(78,), dtype=torch.float32)}
       edata_schemes={})]

In [13]:
train_lg, test_lg = data[0], data[1]

IndexError: list index out of range

In [11]:
train_lg = data[0]

In [13]:
train_lg = dgl.add_self_loop(train_lg)
# test_lg = dgl.add_self_loop(test_lg)

In [12]:
train_lg

Graph(num_nodes=139444, num_edges=1492813740,
      ndata_schemes={'label': Scheme(shape=(), dtype=torch.int64), 'h': Scheme(shape=(78,), dtype=torch.float32)}
      edata_schemes={})

In [14]:
all_index = torch.arange(train_lg.num_nodes())

In [14]:
sampler = dgl.dataloading.MultiLayerFullNeighborSampler(4)
dataloader = dgl.dataloading.DataLoader(
    train_lg, all_index, sampler,
    batch_size=1024, shuffle=True, drop_last=False)

In [18]:
model = GATModel(in_feats=78, hid_feats=10, out_feats=15)

In [23]:
opt = torch.optim.Adam(model.parameters(), lr=0.01)

In [36]:
len(dataloader)

518

In [37]:
# model = torch.load('cic2017_GATModel')

In [24]:
for epoch in range(100):
    for input_nodes, output_nodes, blocks in dataloader:
        model = model.cuda()
        blocks = [b.to(torch.device('cuda')) for b in blocks]
        input_features = blocks[0].srcdata['x']
        output_labels = blocks[-1].dstdata['y']
        output_predictions = model(blocks, input_features)
        loss = F.cross_entropy(output_predictions, output_labels)
        opt.zero_grad()
        loss.backward()
        opt.step()
    print(loss.item())
    f1 = evaluate(model, test_dataloader)
    print('f1=', f1)


NameError: name 'dataloader' is not defined

In [15]:
all_index2 = torch.arange(test_lg.num_nodes())
test_dataloader = dgl.dataloading.DataLoader(
    test_lg, all_index2, sampler,
    batch_size=1024, shuffle=True, drop_last=False)

In [52]:
df = pd.DataFrame(classification_report(indices,labels, digits=4, output_dict=True)).transpose()
df

,precision,recall,f1-score,support
0,0.924092,0.989399,0.955631,283.000000
1,1.000000,1.000000,1.000000,141.000000
2,1.000000,0.985915,0.992908,213.000000
5,1.000000,1.000000,1.000000,15.000000
6,1.000000,0.727273,0.842105,11.000000
7,0.833333,0.833333,0.833333,6.000000
8,0.900000,0.818182,0.857143,11.000000
9,0.993443,0.958861,0.975845,316.000000
10,1.000000,1.000000,1.000000,23.000000
12,0.666667,0.400000,0.500000,5.000000


In [25]:
torch.save(model ,'grapshage_cic2017')

In [15]:
model = torch.load('grapshage_cic2017')

In [15]:
class GATModel(nn.Module):
    def __init__(self, in_feats, hid_feats, out_feats):
        super().__init__()
        self.convs = nn.ModuleList()
        self.depth = k
        for i in range(k):
            self.convs.append(dglnn.GATConv(in_feats=i * hid_feats + in_feats, out_feats=hid_feats, num_heads=1))
        
    def forward(self, blocks, inputs):
        # 输入是节点的特征
        h = inputs
        for i in range(self.depth):
            conv = self.convs[i]
            x = copy.copy(h)
#             h = nn.LayerNorm(h.size(1))(h)
            h = F.relu(h)
            h = conv(graph, h)
            h = h1.view(h.size(0), h.size(1) * h.size(2))
            h = torch.cat([h, x], -1)
        return h
def evaluate(model, dataloader):
    model = model.cpu()
    model.eval()
    with torch.no_grad():
        pred_label = []
        true_label = []
        for input_nodes, output_nodes, blocks in dataloader:
            indices, labels = test(model, blocks, blocks[0].srcdata['x'], blocks[-1].dstdata['y'])
            pred_label.append(indices)
            true_label.append(labels)
        pred_label = np.concatenate(pred_label, 0)
        true_label = np.concatenate(true_label, 0)
        return f1_score(true_label, pred_label, average='macro')

def test(model, graph, features, labels):
    model.eval()
    with torch.no_grad():
        logits = model(graph, features)
        _, indices = torch.max(logits, dim=1)
        indices = indices.numpy()
        labels = labels.numpy()
        return indices, labels

In [27]:
class SAGE(nn.Module):
    def __init__(self, in_feats, hid_feats, out_feats, k):
        super().__init__()
        # 实例化SAGEConve，in_feats是输入特征的维度，out_feats是输出特征的维度，aggregator_type是聚合函数的类型
        self.convs = nn.ModuleList()
        self.depth = k
        for i in range(k):
            self.convs.append(dglnn.SAGEConv(in_feats=i * hid_feats + in_feats, out_feats=hid_feats, aggregator_type='mean'))
        
    def forward(self, graph, inputs):
        h = inputs
        for i in range(self.depth):
            conv = self.convs[i]
            x = copy.copy(h)
            h = nn.LayerNorm(h.size(1))(h)
            h = F.relu(h)
            h = conv(graph, h)
            h = torch.cat([h, x], -1)
        return h

def evaluate(model, graph, features, labels):
    model.eval()
    with torch.no_grad():
        logits = model(graph, features)
        _, indices = torch.max(logits, dim=1)
        correct = torch.sum(indices == labels)
        return correct.item() * 1.0 / len(labels)

In [28]:
model = SAGE(in_feats=78, hid_feats=10, out_feats=15, k=10)

In [34]:
opt = torch.optim.Adam(model.parameters())

In [35]:
train_lg

Graph(num_nodes=459589, num_edges=339778283,
      ndata_schemes={'x': Scheme(shape=(78,), dtype=torch.float32), 'y': Scheme(shape=(), dtype=torch.int64), 'h': Scheme(shape=(10,), dtype=torch.float32)}
      edata_schemes={})

In [31]:
features = train_lg.ndata['x']

In [32]:
test_features = test_lg.ndata['x']
test_label = test_lg.ndata['y']

In [49]:
Counter(train_lg.ndata['y'].numpy())

Counter({1: 89617,
         0: 70000,
         2: 111163,
         9: 161087,
         8: 3849,
         5: 5555,
         10: 7205,
         12: 1055,
         3: 1369,
         7: 4057,
         6: 4128,
         13: 456,
         14: 15,
         4: 25,
         11: 8})

In [41]:
Counter(test_label.numpy())

Counter({9: 69037,
         2: 47641,
         0: 30000,
         1: 38408,
         3: 587,
         10: 3088,
         5: 2380,
         7: 1739,
         8: 1650,
         12: 452,
         6: 1769,
         13: 196,
         14: 6,
         4: 11,
         11: 3})

In [36]:
model.train()
for epoch in range(2000):
    start_time = time.time()
    logits = model(train_lg, features)
    loss = F.cross_entropy(logits, train_lg.ndata['y'])
    acc = evaluate(model, test_lg, test_features, test_label)
    opt.zero_grad()
    loss.backward()
    opt.step()
    end_time = time.time()
    if epoch % 100 == 0:
        torch.save(model ,'cic3_dgl_epoch' + str(epoch))
    print('epoch: {:04d}'.format(epoch),
          'loss_train: {:.4f}'.format(loss),
          'acc_train: {:.4f}'.format(acc),
          'time: {:.4f}s'.format(end_time - start_time))

epoch: 0000 loss_train: 0.4363 acc_train: 0.9285 time: 31.0255s
epoch: 0001 loss_train: 0.4356 acc_train: 0.9278 time: 32.3302s
epoch: 0002 loss_train: 0.4293 acc_train: 0.9289 time: 32.3644s
epoch: 0003 loss_train: 0.4287 acc_train: 0.9293 time: 32.3987s
epoch: 0004 loss_train: 0.4269 acc_train: 0.9301 time: 32.1768s
epoch: 0005 loss_train: 0.4228 acc_train: 0.9303 time: 31.3776s
epoch: 0006 loss_train: 0.4215 acc_train: 0.9309 time: 31.3223s
epoch: 0007 loss_train: 0.4214 acc_train: 0.9317 time: 32.4094s
epoch: 0008 loss_train: 0.4210 acc_train: 0.9320 time: 33.7153s
epoch: 0009 loss_train: 0.4208 acc_train: 0.9319 time: 31.9309s
epoch: 0010 loss_train: 0.4219 acc_train: 0.9313 time: 33.2010s
epoch: 0011 loss_train: 0.4231 acc_train: 0.9314 time: 32.8739s
epoch: 0012 loss_train: 0.4227 acc_train: 0.9319 time: 32.2514s
epoch: 0013 loss_train: 0.4220 acc_train: 0.9333 time: 35.9604s
epoch: 0014 loss_train: 0.4222 acc_train: 0.9332 time: 33.8469s
epoch: 0015 loss_train: 0.4227 acc_train

KeyboardInterrupt: 

In [37]:
model.eval()
logits = model(test_lg, test_lg.ndata['x'])
_, indices = torch.max(logits, dim=1)

In [46]:
Counter(indices.numpy())

Counter({9: 70970,
         2: 50674,
         0: 28306,
         1: 38769,
         3: 314,
         8: 1620,
         5: 1762,
         7: 1102,
         6: 1684,
         33: 757,
         167: 26,
         11: 727,
         28: 255,
         4: 1})

In [44]:
report = classification_report(test_label, indices)

/home/xiaoyujie/anaconda3/envs/densegat2/lib/python3.6/site-packages/sklearn/metrics/_classification.py:1248: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/xiaoyujie/anaconda3/envs/densegat2/lib/python3.6/site-packages/sklearn/metrics/_classification.py:1248: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/xiaoyujie/anaconda3/envs/densegat2/lib/python3.6/site-packages/sklearn/metrics/_classification.py:1248: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start

In [43]:
report

,precision,recall,f1-score,support
0,0.898396,0.847667,0.872294,30000.000000
1,0.980010,0.989221,0.984594,38408.000000
2,0.931148,0.990428,0.959874,47641.000000
3,0.869427,0.465077,0.605993,587.000000
4,1.000000,0.090909,0.166667,11.000000
5,0.652667,0.483193,0.555287,2380.000000
6,0.537411,0.511588,0.524182,1769.000000
7,0.780399,0.494537,0.605421,1739.000000
8,0.845062,0.829697,0.837309,1650.000000
9,0.969226,0.996364,0.982608,69037.000000


In [123]:
report

,precision,recall,f1-score,support
0,0.918919,0.237762,0.377778,143.000000
1,0.976382,0.948613,0.962297,150000.000000
2,0.952000,0.974087,0.962917,150000.000000
3,0.937245,0.968527,0.952629,27325.000000
4,1.000000,0.166667,0.285714,24.000000
accuracy,0.961575,0.961575,0.961575,0.961575
macro avg,0.956909,0.659131,0.708267,327492.000000
weighted avg,0.961926,0.961575,0.961470,327492.000000


In [124]:
confusion_matrix(test_lg.ndata['y'], indices)

array([[    34,      4,     54,     51,      0],
       [     0, 142292,   6876,    832,      0],
       [     0,   3001, 146113,    886,      0],
       [     2,    437,    421,  26465,      0],
       [     1,      0,     16,      3,      4]])

In [93]:
report

,precision,recall,f1-score,support
0,0.988664,0.972483,0.980507,60000.000000
1,0.999427,0.999141,0.999284,38408.000000
2,0.991440,0.999202,0.995306,47641.000000
3,0.909710,0.909710,0.909710,587.000000
4,0.068966,0.181818,0.100000,11.000000
5,0.987516,0.997059,0.992264,2380.000000
6,0.870229,0.773318,0.818916,1769.000000
7,0.980460,0.981024,0.980742,1739.000000
8,0.975802,0.977576,0.976688,1650.000000
9,0.988701,0.998740,0.993695,69037.000000


In [87]:
report

,precision,recall,f1-score,support
0,0.991160,0.969833,0.980381,60000.000000
1,0.999349,0.999141,0.999245,38408.000000
2,0.990966,0.999244,0.995088,47641.000000
3,0.905245,0.911414,0.908319,587.000000
4,0.068966,0.181818,0.100000,11.000000
5,0.989162,0.997059,0.993095,2380.000000
6,0.858502,0.816280,0.836859,1769.000000
7,0.978223,0.981599,0.979908,1739.000000
8,0.976378,0.976970,0.976674,1650.000000
9,0.987310,0.999609,0.993421,69037.000000
